
# Download Swot from AVISO
This notebook explains how to explore and retrieve available SWOT LR data from AVISO's THREDDS Data Server and FTP Server, presented [here](https://www.aviso.altimetry.fr/en/missions/current-missions/swot/access-to-data.html).

L2 data can be explored at: https://tds.aviso.altimetry.fr/thredds/L2/L2-SWOT-DATA/L2-SWOT.html

L3 data can be explored at: https://tds.aviso.altimetry.fr/thredds/L3/dataset-l3-swot-karin-nadir-pre-validated.html


<div class="alert alert-danger"><h4>Warning</h4><p>This tutorial is dedicated to **external use** of Swot data only, it cannot be ran inside CNES infrastructure because of the technical proxy.</p></div>



<div class="alert alert-info"><h4>Note</h4><p>To learn how to subset and download unsmoothed (250-m) Swot LR data, please consult this tutorial:
   `sphx_glr_tutorials_auto_tutorials_external_subset_unsmoothed_aviso.py`</p></div>



## Tutorial Objectives
* Discover Aviso's Thredds Data Server catalogue
* Download files through HTTP and FTP
* Download data subset through Opendap



## Import + code



In [1]:
import os
import re
from getpass import getpass
import requests as rq
import ftplib
import xarray as xr
from xarray.backends import PydapDataStore

In [2]:
def _get_filename_from_cd(cd):
    """ Get filename from content-disposition """
    if not cd:
        return None
    fname = re.findall('filename="(.+)"', cd)
    if len(fname) == 0:
        return None
    return fname[0]

def http_download_file(dataset_url:str, output_path:str):
    response = rq.get(dataset_url, auth=(username, password))
    
    filename = _get_filename_from_cd(response.headers['content-disposition'])
    file_path = os.path.join(output_path, filename)
    
    open(file_path, 'wb').write(response.content)
    return file_path

def _download_file(ftp:str, filename:str, target_directory:str):
    try:
        local_filepath = os.path.join(target_directory, filename)
        with open(local_filepath, 'wb') as file:
            ftp.retrbinary('RETR %s' % filename, file.write)
            print(f"Downloaded {filename} to {local_filepath}")
            return local_filepath
    except Exception as e:
        print(f"Error downloading {filename}: {e}")
        
def ftp_download_file(ftp_path:str, filename:str, output_path:str):
    # Set up FTP server details
    ftpAVISO = 'ftp-access.aviso.altimetry.fr'
    try:
        # Logging into FTP server using provided credentials
        with ftplib.FTP(ftpAVISO) as ftp:
            ftp.login(username, password)
            ftp.cwd(ftp_path)
            print(f"Connection Established {ftp.getwelcome()}")

            # Check if the file exists in the directory
            if filename in ftp.nlst():
                return _download_file(ftp, filename, output_path)
            else:
                print(f"File {filename} does not exist in the directory {ftp}.")
    except ftplib.error_perm as e:
        print(f"FTP error: {e}")
    except Exception as e:
        print(f"Error: {e}")


def open_opendap_dataset(dataset_url):
    session = rq.Session()
    session.auth = (username, password)
    store = PydapDataStore.open(dataset_url, session=session, timeout=300, user_charset='UTF-8')
    return xr.open_dataset(store)

## Parameters
Define output folder to save results



In [3]:
output_dir = "downloads"

### Authentication parameters
Enter your AVISO+ credentials



In [7]:
password = getpass(f"Enter password for {username}:")

Enter password for cyril.germineaud@cnes.fr: ········


In [ ]:
username = input("Enter username:")

## Download files through HTTP
To find the data you want to download, you can explore the [Swot LR L2](https://tds.aviso.altimetry.fr/thredds/L2/L2-SWOT-DATA/L2-SWOT.html) or [Swot LR L3](https://tds.aviso.altimetry.fr/thredds/L3/dataset-l3-swot-karin-nadir-pre-validated.html) catalogues.
Copy the **FileServer** url to download data through HTTP : 

<img src="file://../../../_static/tutorials/TDS_file_server.png">



### Swot LR L2 data



In [ ]:
l2_dataset_url = "https://tds.aviso.altimetry.fr/thredds/fileServer/dataset-l2-swot-karin-lr-ssh-pre-validated/PIC0/Basic/cycle_014/SWOT_L2_LR_SSH_Basic_014_001_20240417T111950_20240417T121118_PIC0_01.nc"

In [ ]:
l2_file = http_download_file(l2_dataset_url, output_dir)
l2_file

### Swot LR L3 data



In [ ]:
l3_dataset_url = "https://tds.aviso.altimetry.fr/thredds/fileServer/dataset-l3-swot-karin-nadir-pre-validated/l3_lr_ssh/v1_0/Basic/cycle_014/SWOT_L3_LR_SSH_Basic_014_001_20240417T111950_20240417T121116_v1.0.nc"

In [ ]:
l3_file = http_download_file(l3_dataset_url, output_dir)
l3_file

## Download files through FTP
Files available through FTP follow the same file tree as in the THREDDS Data Server.

.. list-table:: Discover data on AVISO FTP
  :width: 130%
  :widths: auto
  :header-rows: 1

  * - Data
    - FTP starting point
    - Browsing files URL
  * - Swot LR L2
    - */swot_products/l2_karin/l2_lr_ssh*
    - https://tds.aviso.altimetry.fr/thredds/catalog/dataset-l2-swot-karin-lr-ssh-pre-validated/catalog.html
  * - Swot LR L3
    - */swot_products/l3_karin_nadir/l3_lr_ssh*
    - https://tds.aviso.altimetry.fr/thredds/catalog/dataset-l3-swot-karin-nadir-pre-validated/catalog.html



### Swot LR L2 data



In [ ]:
l2_ftp_path = '/swot_products/l2_karin/l2_lr_ssh/PGC0/WindWave/cycle_001/'
l2_ftp_file = 'SWOT_L2_LR_SSH_WindWave_001_155_20230726T173636_20230726T182805_PGC0_01.nc'

In [ ]:
l2_file = ftp_download_file(l2_ftp_path, l2_ftp_file, output_dir)
l2_file

### Swot LR L3 data



In [ ]:
l3_ftp_path = '/swot_products/l3_karin_nadir/l3_lr_ssh/v1_0/Basic/cycle_011/'
l3_ftp_file = 'SWOT_L3_LR_SSH_Basic_011_550_20240305T114902_20240305T124029_v1.0.nc'

In [ ]:
l3_file = ftp_download_file(l3_ftp_path, l3_ftp_file, output_dir)
l3_file

## Load subset via OPeNDAP
To find the data you want to subset, you can explore the [Swot LR L2](https://tds.aviso.altimetry.fr/thredds/L2/L2-SWOT-DATA/L2-SWOT.html) or [Swot LR L3](https://tds.aviso.altimetry.fr/thredds/L3/dataset-l3-swot-karin-nadir-pre-validated.html) catalogues.

Copy the **OPeNDAP** url to subset data through Opendap : 

<img src="file://../../../_static/tutorials/TDS_opendap.png">



### Swot LR L2 data



In [ ]:
l2_dataset_url = "https://tds.aviso.altimetry.fr/thredds/dodsC/dataset-l2-swot-karin-lr-ssh-pre-validated/PIC0/Basic/cycle_014/SWOT_L2_LR_SSH_Basic_014_001_20240417T111950_20240417T121118_PIC0_01.nc"

In [ ]:
l2_dataset = open_opendap_dataset(l2_dataset_url)
l2_dataset

#### Select variables



In [ ]:
variables = ['time', 'ssha_karin_2', 'mean_sea_surface_cnescls']

In [ ]:
ds = l2_dataset[variables]
ds

#### Select lines
Select lines to load



In [ ]:
idx_first, idx_last = 5000, 6000

In [ ]:
ds = xr.merge([ds[var][idx_first:idx_last] for var in variables])

In [ ]:
ds.load()

### Swot LR L3 data



In [ ]:
l3_dataset_url = "https://tds.aviso.altimetry.fr/thredds/dodsC/dataset-l3-swot-karin-nadir-pre-validated/l3_lr_ssh/v1_0/Basic/cycle_014/SWOT_L3_LR_SSH_Basic_014_350_20240429T223450_20240429T232617_v1.0.nc"

In [ ]:
l3_dataset = open_opendap_dataset(l3_dataset_url)
l3_dataset

#### Select variables



In [ ]:
variables = ['time', 'ssha']

In [ ]:
ds = l3_dataset[variables]
ds

#### Select lines
Select lines to load



In [ ]:
idx_first, idx_last = 5000, 6000

In [ ]:
ds = xr.merge([ds[var][idx_first:idx_last] for var in variables])

In [ ]:
ds.load()

### Store subset to NetCDF file



In [ ]:
ds.to_netcdf(os.path.join(output_dir, 'dataset.nc'))